In [36]:
# Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from datetime import datetime
import requests
import pandas_datareader as pdr
from datetime import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
# Function to Validate Date
def validateShift(shift):
    shift = int(shift)
    # Try Check on Format
    try:
        if 0 > shift <= 365:
            pass
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Please enter a whole number.")

In [3]:
# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")

# Validate Correct Date Format
validate(date)

Pick a start date for your analysis (YYYY-MM-DD):  2016-01-01


In [4]:
# Pick a Stock
stock_selection = input("Pick a stock ticker for your analysis: ")

# Validate Correct Date Format
validateTick(stock_selection)

Pick a stock ticker for your analysis:  CME


In [5]:
# Pick Shift
shift_selection = input("Pick how many days to shift your close values for your analysis: ")

# Validate Correct Date Format
validateShift(shift_selection)

Pick how many days to shift your close values for your analysis:  1


In [6]:
# Function to Create Stocks
def stockCreation(date, stock_selection, days_shifted):
    
    # Read Stock Data
    globals()[stock_selection] = pdr.DataReader(f'{stock_selection}','yahoo',date)
    
    # Create Shifted Close Column
    globals()[stock_selection]["Shifted Close"] = globals()[stock_selection]["Adj Close"].shift(-int(days_shifted))
    
    # Drop Rows Without Shift Values
    globals()[stock_selection].drop(globals()[stock_selection].tail(int(days_shifted)).index, inplace = True) 
    
    # Empty List to Hold Profit or Loss
    profit_loss = []
    
    # Iterate Through DataFrame
    for x in range(len(globals()[stock_selection])):
        
        # Handle Profitable Days
        if globals()[stock_selection]["Shifted Close"][x] > globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Profit" to Empty List
            profit_loss.append(1)
            
        # Handle Loss Days
        elif globals()[stock_selection]["Shifted Close"][x] < globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Loss" to Empty List
            profit_loss.append(0)
            
        # Handle Push Days
        elif globals()[stock_selection]["Shifted Close"][x] == globals()[stock_selection]["Adj Close"][x]:
            
            # Append "Push" to Empty List
            profit_loss.append(2)
    
    # Create Profit/Loss Column
    globals()[stock_selection]["Profit_Loss"] = profit_loss
    
    # Create Returns Column
    globals()[stock_selection]['Returns'] = np.log(globals()[stock_selection]['Adj Close'] / globals()[stock_selection]['Adj Close'].shift(1))
    
    # Iterate to Create STD Columns
    for x in range(5, 20, 5):
        
        # Create STD Column Based on Iteration
        col_name = 'std_' + str(x)
        
        # Add STD Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).std()

    # Iterate to Create MA Columns
    for x in range(10, 30, 5):
        
        # Create MA Column Based on Iteration
        col_name = 'ma_' + str(x)
        
        # Add MA Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].rolling(window=x).mean()

    # Iterate to Create % Change Columns
    for x in range(3, 12, 3):
        
        # Create % Change Column Based on Iteration
        col_name = 'pct_' + str(x)
        
        # Add % Change Column to DataFrame
        globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'].pct_change().rolling(window=x).sum()

    # Create VMA 4 Column
    col_name = 'vma_4'
    
    # Add VMA 4 Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Volume'].rolling(4).mean()

    # Create Intraday Column
    col_name = 'co'
    
    # Add Intraday Column to DataFrame
    globals()[stock_selection][col_name] = globals()[stock_selection]['Adj Close'] - globals()[stock_selection]['Open']
    
    # Drop All NaN's
    globals()[stock_selection].dropna(inplace=True)
    
    # Return DataFrame
    return globals()[stock_selection]

In [7]:
# Create Stock
data = stockCreation(date, stock_selection, shift_selection)
data.tail()

,High,Low,Open,Close,Volume,Adj Close,Shifted Close,Profit_Loss,Returns,std_5,...,std_15,ma_10,ma_15,ma_20,ma_25,pct_3,pct_6,pct_9,vma_4,co
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-11,186.289993,179.690002,185.759995,182.710007,2515000.0,182.710007,184.369995,1,-0.013536,4.337415,...,4.799721,188.972002,186.876000,187.8665,190.1680,-0.047511,-0.048719,0.006921,2325175.0,-3.049988
2021-02-12,184.589996,182.110001,182.860001,184.369995,1441400.0,184.369995,189.410004,1,0.009044,4.345501,...,4.786531,189.235001,186.898000,187.3780,189.5904,-0.039625,-0.047655,-0.015357,2167750.0,1.509995
2021-02-16,189.899994,183.000000,185.089996,189.410004,2677700.0,189.410004,191.139999,1,0.026969,3.831152,...,4.773276,189.432001,187.234001,187.2575,189.1796,0.022977,-0.009261,-0.012401,2330600.0,4.320007
2021-02-17,191.899994,187.220001,189.419998,191.139999,1513800.0,191.139999,193.990005,1,0.009092,3.555521,...,4.769367,189.345001,187.726667,187.2045,188.8676,0.045555,-0.001956,-0.003164,2036975.0,1.720001
2021-02-18,194.240005,189.720001,190.050003,193.990005,1497200.0,193.990005,195.070007,1,0.014800,4.700105,...,4.329455,189.545001,188.740001,187.4125,188.5676,0.051381,0.011755,0.003726,1782525.0,3.940002


In [8]:
data.info

<bound method DataFrame.info of                   High         Low        Open       Close     Volume  \
Date                                                                    
2016-02-08   90.669998   87.000000   88.410004   90.190002  2758100.0   
2016-02-09   93.809998   89.050003   89.050003   93.309998  3233500.0   
2016-02-10   93.489998   90.440002   93.029999   90.529999  2133700.0   
2016-02-11   90.550003   88.320000   88.360001   89.320000  1952800.0   
2016-02-12   91.290001   89.510002   90.919998   91.150002  1691200.0   
...                ...         ...         ...         ...        ...   
2021-02-11  186.289993  179.690002  185.759995  182.710007  2515000.0   
2021-02-12  184.589996  182.110001  182.860001  184.369995  1441400.0   
2021-02-16  189.899994  183.000000  185.089996  189.410004  2677700.0   
2021-02-17  191.899994  187.220001  189.419998  191.139999  1513800.0   
2021-02-18  194.240005  189.720001  190.050003  193.990005  1497200.0   

             Adj C

<h1>LSTM<h1>

In [9]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [10]:
window_size = 100

feature_column = 2
target_column = 2
X, y = window_data(data, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[88.41000366 89.05000305 93.02999878 88.36000061 90.91999817 92.25
  91.68000031 91.97000122 91.43000031 94.         92.04000092 91.48000336
  92.97000122 93.63999939 92.44999695 91.79000092 92.98999786 93.58000183
  94.76999664 93.94999695 92.12000275 93.44999695 94.44000244 93.93000031
  94.66999817 95.40000153 96.08999634 96.19000244 96.66000366 95.94999695
  95.77999878 95.76999664 96.19000244 95.16999817 95.         95.62000275
  95.69000244 95.81999969 96.87000275 97.47000122 94.22000122 93.73999786
  90.         89.66999817 89.98999786 90.81999969 90.58000183 91.15000153
  91.38999939 91.36000061 92.66999817 93.06999969 93.05000305 93.91000366
  94.12999725 93.86000061 92.68000031 91.75       92.59999847 93.20999908
  92.36000061 92.75       92.33999634 92.91999817 93.91999817 94.37999725
  93.98000336 95.         94.15000153 94.94000244 94.5        96.5
  95.90000153 96.69999695 96.94999695 98.09999847 96.44000244 97.97000122
  98.62000275 97.79000092 98.05999

In [11]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [12]:
# scaling data for LSTM model
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [13]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.00036856]
  [0.00508589]
  [0.03442175]
  [0.        ]
  [0.00930336]
  [0.01920216]
  [0.01495983]
  [0.01711821]
  [0.01309915]
  [0.03222687]
  [0.0176392 ]
  [0.01347131]
  [0.0245609 ]
  [0.02954749]
  [0.02069067]
  [0.01577853]
  [0.02470973]
  [0.02910095]
  [0.03795771]
  [0.03185471]
  [0.01823463]
  [0.02813336]
  [0.03550167]
  [0.03170588]
  [0.03721346]
  [0.04264665]
  [0.04778206]
  [0.04852638]
  [0.05202445]
  [0.04674009]
  [0.04547485]
  [0.04540041]
  [0.04852638]
  [0.0409348 ]
  [0.03966956]
  [0.04428405]
  [0.04480503]
  [0.04577256]
  [0.05358741]
  [0.05805301]
  [0.03386427]
  [0.03029175]
  [0.0024561 ]
  [0.        ]
  [0.        ]
  [0.00179841]
  [0.        ]
  [0.        ]
  [0.00022612]
  [0.        ]
  [0.00695493]
  [0.00997883]
  [0.00982766]
  [0.01632903]
  [0.01799212]
  [0.01595102]
  [0.00703054]
  [0.        ]
  [0.00197435]
  [0.00660644]
  [0.0001519 ]
  [0.0031134 ]
  [0.        ]
  [0.        ]
  [0.00550333]
  

In [14]:
# designing model
model = Sequential()

number_units = 50 # no. of hidden units
dropout_fraction=0.2 # means 20% of layers will be dropped

# layer 1
model.add(LSTM(
    units = number_units, return_sequences = True,
    input_shape = (X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences = True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))


In [15]:
# compiling the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [16]:
# summarizing the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 50)           10400     
_________________________________________________________________
dropout (Dropout)            (None, 100, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 5

In [17]:
# training the model
model.fit(X_train, y_train, epochs= 50, shuffle=False, batch_size=5, verbose=1)

Epoch 1/50
164/164 [==============================] - 10s 34ms/step - loss: 0.0030
Epoch 2/50
164/164 [==============================] - 6s 38ms/step - loss: 0.0296
Epoch 3/50
164/164 [==============================] - 7s 41ms/step - loss: 0.0539
Epoch 4/50
164/164 [==============================] - 7s 41ms/step - loss: 0.0650
Epoch 5/50
164/164 [==============================] - 7s 41ms/step - loss: 0.0906
Epoch 6/50
164/164 [==============================] - 7s 41ms/step - loss: 0.0517
Epoch 7/50
164/164 [==============================] - 7s 43ms/step - loss: 0.0153
Epoch 8/50
164/164 [==============================] - 7s 43ms/step - loss: 0.0051
Epoch 9/50
164/164 [==============================] - 7s 42ms/step - loss: 0.0032
Epoch 10/50
164/164 [==============================] - 7s 43ms/step - loss: 0.0027
Epoch 11/50
164/164 [==============================] - 7s 43ms/step - loss: 0.0031
Epoch 12/50
164/164 [==============================] - 7s 43ms/step - loss: 0.0020
Epoch 13/50


In [18]:
# evaluating the model
model.evaluate(X_test, y_test)

11/11 [==============================] - 1s 12ms/step - loss: 0.0093


0.009273519739508629

In [19]:
# predicting
y_pred = model.predict(X_test)

In [20]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(y_pred)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [21]:
# comparing the real price against the predicted price 
stock = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = data.index[-len(real_prices): ])
stock.tail()

,Real,Predicted
Date,,
2021-02-11,185.759995,202.347336
2021-02-12,182.860001,200.174911
2021-02-16,185.089996,197.490829
2021-02-17,189.419998,195.862686
2021-02-18,190.050003,196.370331


In [23]:
# Root mean square error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"the RMSE for LSTM is {rmse}.")

the RMSE for LSTM is 0.09629911613278112.
